In [1]:
import requests
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String
import json
import time

In [2]:
engine = create_engine("sqlite:///vacancy.db")
Base = declarative_base()

C:\Users\ilove\AppData\Local\Temp\ipykernel_7448\3642724841.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
class Vacancy(Base):
    __tablename__ = 'py_middle'
    id = Column(Integer, primary_key=True, autoincrement=True)
    position  = Column(String(256))
    company_name = Column(String(256))
    job_description  = Column(String(1024))
    key_skills = Column(String(256))

In [4]:
Base.metadata.create_all(engine)

In [18]:
url = 'https://api.hh.ru/vacancies'
user_agent = {'User-agent': 'Mozilla/5.0'}
params = {"page": 1, "per_page": 20, "search_field":"name" ,"text": "middle python developer" }

In [19]:
result = requests.get(url, headers=user_agent, params=params)

In [20]:
result.status_code

200

In [21]:
vacancies  = json.loads(result.text)


In [9]:
pages = vacancies["pages"]
#pages = 7

In [23]:
session = Session(bind=engine, expire_on_commit=False)
for n in range(1, pages):
    params = {"page": n, "per_page": 20,"search_field":"name" ,"text": "middle python developer" }
    result = requests.get(url, headers=user_agent, params=params)
    vacancies  = json.loads(result.text)
    vacancies = vacancies["items"]
    print(vacancies)
    for vac in vacancies:
        vac_url = "https://api.hh.ru/vacancies/"+str(vac["id"])
        #print(vac_url)
        res =  requests.get(vac_url, headers=user_agent)

        if (result.status_code == 200):
            vacancy  = json.loads(res.text)
            print(vacancy)
            position = vacancy["name"]
            company_name = vacancy["employer"]["name"]
            job_description = vacancy["description"]
            key_skills = list()
            for j in vacancy["key_skills"]:
                key_skills.append(j["name"])
            key_skills = str(key_skills)
            vac = Vacancy(position=position, company_name=company_name, job_description=job_description, key_skills=key_skills)
            session.add(vac)
            time.sleep(0.1)
        else:
            print("error")
            continue
    session.commit()
    print("page "+str(n)+" done!")
session.close()

[{'id': '82778051', 'premium': False, 'name': 'Middle Python(django) разработчик', 'department': None, 'has_test': False, 'response_letter_required': False, 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'}, 'salary': None, 'type': {'id': 'open', 'name': 'Открытая'}, 'address': {'city': 'Москва', 'street': 'Смоленская площадь', 'building': '5', 'lat': 55.748285, 'lng': 37.581415, 'description': None, 'raw': 'Москва, Смоленская площадь, 5', 'metro': {'station_name': 'Смоленская', 'line_name': 'Арбатско-Покровская', 'station_id': '3.131', 'line_id': '3', 'lat': 55.747713, 'lng': 37.583802}, 'metro_stations': [{'station_name': 'Смоленская', 'line_name': 'Арбатско-Покровская', 'station_id': '3.131', 'line_id': '3', 'lat': 55.747713, 'lng': 37.583802}, {'station_name': 'Смоленская', 'line_name': 'Филевская', 'station_id': '4.132', 'line_id': '4', 'lat': 55.749083, 'lng': 37.582173}], 'id': '13627117'}, 'response_url': None, 'sort_point_distance': None, 'published_at'